# 연관분석

- 장바구니분석 또는 서열분석
- 기업의 DB에서 상품의 구매, 서비스 등 일련의 거래 또는 사건들 간의 규칙을 발견하기 위해 적용
***
- 조건과 반응의 형태(if - then)
- if A then B -> A가 일어나면 B가 일어난다.

예를 들어 "아메리카노를 마시는 손님 중 10%가 브라우니를 먹는다."

### 연관분석의 측도

#### 1. 지지도(Support)
- 전체 거래 중 항목 A와 B를 동시에 포함하는 거래의 비율

#### 2. 신뢰도(Confidence)
- 항목 A를 포함한 거래 중에서 항목 A와 항목 B가 같이 포함될 확률, 연관성의 정도
- 지지도 / P(A)

#### 3. 향상도(Lift)
- A를 구매하지 않았을 때 품목 B의 구매확률에 비해 A가 구매됐을 때 품목 B의 구매확률의 증가 비. 연관규칙 A->B는 품목 A와 B의 구매가 서로 관련이 없는 경우에 향상도가 1이 된다.
- 신뢰도 / P(B)
- 1보다 큰 경우는 우연적 기회보다 높은 확률 / 1인 경우는 독립 / 1보다 작은 경우는 우연적 기회보다 낮은 확률
***
### 연관규칙의 절차

- 최소 지지도 보다 큰 집합만을 대상으로 높은 지지도를 갖는 품목 집합을 찾는다.
- 5%에서 시작하여 규칙이 충분히 도출되는지를 보고 다양하게 조절하여 시도
***
### 장/단점
#### 장점
- 탐색적인 기법으로 분석 결과를 쉽게 이해할 수 있다.
- 강력한 비목적성 분석기법으로 유용하게 활용이 가능하다.
- 분석을 위한 계산이 간단하다.

#### 단점
- 품목수가 증가하면 분석에 필요한 계산은 기하급수적으로 늘어난다. -> 유사 품목을 재범주화 혹은 신뢰도 하한을 새롭게 정의하고 진행
- 너무 세분화된 품목이 있을 경우 규칙 발견시 의미없는 분석이 될 수 있다.
- 거래량이 적은 품목은 규칙 발견시 제외되기 쉽다.

In [80]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv('chipotle.tsv', engine = 'python', delimiter = '\t')

In [14]:
data.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [24]:
data['item_name'] = [[x] for x in data.item_name]

In [37]:
item_df = data.groupby('order_id').item_name.sum().reset_index(name = 'item_list')
item_df.head()

,order_id,item_list
0,1,"[Chips and Fresh Tomato Salsa, Izze, Nantucket..."
1,2,[Chicken Bowl]
2,3,"[Chicken Bowl, Side of Chips]"
3,4,"[Steak Burrito, Steak Soft Tacos]"
4,5,"[Steak Burrito, Chips and Guacamole]"


In [38]:
item_df['item_list'] = [list(set(x)) for x in item_df.item_list]

In [39]:
item_df.sample(10)

,order_id,item_list
1627,1628,"[Steak Burrito, Chicken Burrito]"
339,340,"[Chips and Guacamole, Chicken Burrito]"
1676,1677,"[Veggie Salad Bowl, Chips and Guacamole]"
576,577,"[Bottled Water, Chips and Guacamole, Chicken B..."
539,540,"[Steak Burrito, Chips and Guacamole]"
906,907,"[Chips and Guacamole, Steak Soft Tacos, Chips,..."
160,161,"[Chicken Burrito, Chicken Bowl]"
766,767,"[Steak Bowl, Chips]"
880,881,"[Chicken Burrito, Chicken Bowl]"
1647,1648,"[Steak Burrito, Chicken Crispy Tacos, Chips an..."


## MLXTEND 이용

In [36]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [40]:
te = TransactionEncoder()
te_array = te.fit_transform(item_df.item_list)
df = pd.DataFrame(te_array, columns = te.columns_)

In [41]:
df.sample(5)

,6 Pack Soft Drink,Barbacoa Bowl,Barbacoa Burrito,Barbacoa Crispy Tacos,Barbacoa Salad Bowl,Barbacoa Soft Tacos,Bottled Water,Bowl,Burrito,Canned Soda,...,Steak Crispy Tacos,Steak Salad,Steak Salad Bowl,Steak Soft Tacos,Veggie Bowl,Veggie Burrito,Veggie Crispy Tacos,Veggie Salad,Veggie Salad Bowl,Veggie Soft Tacos
1093,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
1715,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
835,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
305,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1357,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### 최소 지지도가 0.05 이상인 규칙 집합

In [49]:
freq_items = apriori(df, min_support = 0.05, use_colnames = True)
freq_items.sample(10)

,support,itemsets
7,0.113413,(Chips)
11,0.102508,(Steak Bowl)
0,0.083969,(Bottled Water)
16,0.062159,"(Chips and Guacamole, Chicken Burrito)"
1,0.051254,(Canned Soda)
2,0.150491,(Canned Soft Drink)
6,0.058342,(Chicken Soft Tacos)
4,0.266630,(Chicken Burrito)
9,0.258451,(Chips and Guacamole)
15,0.081243,"(Chips and Guacamole, Chicken Bowl)"


Chips and Guacamole와 Chicken Burrito를 같이 구매할 확률은 0.062159이다.

Canned Soft Drink를 구매할 확률은 0.150491이다.

In [45]:
association_rules(freq_items, metric = "confidence", min_threshold = .05)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Canned Soft Drink),(Chicken Bowl),0.150491,0.335333,0.060523,0.402174,1.199328,0.010059,1.111807
1,(Chicken Bowl),(Canned Soft Drink),0.335333,0.150491,0.060523,0.180488,1.199328,0.010059,1.036604
2,(Chips),(Chicken Bowl),0.113413,0.335333,0.066521,0.586538,1.749124,0.028490,1.607568
3,(Chicken Bowl),(Chips),0.335333,0.113413,0.066521,0.198374,1.749124,0.028490,1.105985
4,(Chips and Guacamole),(Chicken Bowl),0.258451,0.335333,0.081243,0.314346,0.937416,-0.005424,0.969392
5,(Chicken Bowl),(Chips and Guacamole),0.335333,0.258451,0.081243,0.242276,0.937416,-0.005424,0.978653
6,(Chips and Guacamole),(Chicken Burrito),0.258451,0.266630,0.062159,0.240506,0.902022,-0.006752,0.965603
7,(Chicken Burrito),(Chips and Guacamole),0.266630,0.258451,0.062159,0.233129,0.902022,-0.006752,0.966979


#### 최소 신뢰도가 0.5 이상인 집합으로 lift(향상도)가 1보다 클수록 우연히 일어나지 않았다는 의미다. 아무런 관계가 없을 경우 1로 표시된다. 

#### Canned Soft Drink과 Chicken Bowl이 함께 등장할 확률(지지도)은 0.060523으로 약 6%이다. 이 때 향상도는 1.199328로 Chicken Bowl만 구매할때 보다 Canned Soft Drink와 Chicken Bowl이 함께 구매될 확률이 1.19배 높다는 것을 의미한다. 

#### conviction = 찾아낸 규칙이 얼마나 잘못되었는지를 확인
- 값이 너무 높은 경우 consequent가 antecedent에 의존성이 높다. 의존성이 없을 경우(향상도 = 1) 이 값도 1

#### leverage = 두 품목의 지지도 - 각 품목의 지지도 곱
- ex > Canned Soft Drink의 support = 0.150491 / Chicken Bowl의 support = 0.335333 / 두 품목의 support = 0.060523으로 leverage 값은 0.060523 - (0.150491 * 0.335333) = 0.010059

In [56]:
0.060523 - 0.150491 * 0.335333

0.010058401496999998

***
## APYORI 이용

In [57]:
from apyori import apriori

In [117]:
association_rules = apriori(item_df.item_list, min_support = .05, min_confidence = .0005)
association_result = list(association_rules)

전체 주문에서 5% 이상의 주문 접수가 된 품목 탐색

In [118]:
rules_1 = pd.DataFrame(association_result)

In [119]:
rules_1

,items,support,ordered_statistics
0,(Bottled Water),0.083969,"[((), (Bottled Water), 0.08396946564885496, 1.0)]"
1,(Canned Soda),0.051254,"[((), (Canned Soda), 0.05125408942202835, 1.0)]"
2,(Canned Soft Drink),0.150491,"[((), (Canned Soft Drink), 0.1504907306434024,..."
3,(Chicken Bowl),0.335333,"[((), (Chicken Bowl), 0.33533260632497275, 1.0)]"
4,(Chicken Burrito),0.266630,"[((), (Chicken Burrito), 0.26663031624863687, ..."
5,(Chicken Salad Bowl),0.053435,"[((), (Chicken Salad Bowl), 0.0534351145038167..."
6,(Chicken Soft Tacos),0.058342,"[((), (Chicken Soft Tacos), 0.0583424209378407..."
7,(Chips),0.113413,"[((), (Chips), 0.1134133042529989, 1.0)]"
8,(Chips and Fresh Tomato Salsa),0.059978,"[((), (Chips and Fresh Tomato Salsa), 0.059978..."
9,(Chips and Guacamole),0.258451,"[((), (Chips and Guacamole), 0.258451472191930..."


In [120]:
print(f'연관분석에서 생성된 규칙의 개수는 {rules_1.shape[0]}개이다.')

연관분석에서 생성된 규칙의 개수는 17개이다.


In [121]:
rules_1['lhs'] = [x[0][0] for x in rules_1.ordered_statistics] # 좌측항 생성

In [122]:
rules_1['rhs'] = [x[0][1] for x in rules_1.ordered_statistics] # 우측항 생성

In [123]:
rules_1['confindence'] = [x[0][2] for x in rules_1.ordered_statistics] # 신뢰도 생성

In [124]:
rules_1['lift'] = [x[0][3] for x in rules_1.ordered_statistics] # 향상도 생성

In [125]:
rules_1['n_items'] = rules_1['items'].apply(lambda x : len(x))

In [126]:
rules_1.drop(['ordered_statistics', 'items'], axis = 1, inplace = True) # 불필요한 열 삭제

In [127]:
rules_1.head()

,support,lhs,rhs,confindence,lift,n_items
0,0.083969,(),(Bottled Water),0.083969,1.0,1
1,0.051254,(),(Canned Soda),0.051254,1.0,1
2,0.150491,(),(Canned Soft Drink),0.150491,1.0,1
3,0.335333,(),(Chicken Bowl),0.335333,1.0,1
4,0.266630,(),(Chicken Burrito),0.266630,1.0,1


In [128]:
rules_1[rules_1.n_items > 1]

,support,lhs,rhs,confindence,lift,n_items
13,0.060523,(),"(Canned Soft Drink, Chicken Bowl)",0.060523,1.0,2
14,0.066521,(),"(Chips, Chicken Bowl)",0.066521,1.0,2
15,0.081243,(),"(Chips and Guacamole, Chicken Bowl)",0.081243,1.0,2
16,0.062159,(),"(Chips and Guacamole, Chicken Burrito)",0.062159,1.0,2


향상도가 1로 나온 이유는 이전 구매 품목(lhs)가 빈칸으로 되어있고 이후 구매 품목(rhs)에는 품목이 존재해서 lhs, rhs가 서로 독립이기 때문이다.